![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

In [9]:
#Importação de pacotes
import pandas as pd  # Series and DataFrames
import numpy as np  # linear algebra
import matplotlib
import matplotlib.pyplot as plt
import sweetviz as sv
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go 
import math

# nltk
import nltk
import ast
import random
import warnings
 


from matplotlib import cm
from plotly.subplots import make_subplots 
from sklearn import preprocessing
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError   
from sklearn.model_selection import train_test_split
from dython.nominal import associations

#from pandas_profiling import ProfileReport
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

pd.set_option('display.max_columns', None)

# visualization
from wordcloud import WordCloud
colors = ['#494BD3', '#E28AE2', '#F1F481', '#79DB80', '#DF5F5F',
              '#69DADE', '#C2E37D', '#E26580', '#D39F49', '#B96FE3']
from plotly.offline import init_notebook_mode, iplot

from dython.nominal import associations

pd.set_option('display.max_columns', None)
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format




In [10]:
from PIL import Image
from IPython.display import display

In [11]:
from sklearn import __version__ as sklearn_version
print('Installed version for scikit-learn: {}.'.format(sklearn_version))

Installed version for scikit-learn: 1.0.2.


In [12]:
# Informa a URL de importação do dataset
# Função para ler os datasets
# OBS: o caminho deverá serguir o caminho do diretorio que está na sua maquina configurado. No meu, está dentro da pasta "data/data_dsmarket/"

def read_dataset():
    try:
        # Caminhos corretos para os arquivos CSV
        dsales = pd.read_csv("data/data_dsmarket/item_sales.csv")## , index_col = 0
        print('"item_sales.csv" imported successfully!')
        # Exibir as primeiras linhas do dataframe
        

        dprices = pd.read_csv("data/data_dsmarket/item_prices.csv") ## , index_col = 0
        print('"item_prices.csv" imported successfully!')
        # Exibir as primeiras linhas do dataframe
      

        dcalendarios_eventos = pd.read_csv("data/data_dsmarket/daily_calendar_with_events.csv") ## ,index_col = 0
        print('"daily_calendar_with_events.csv" imported successfully!')
        # Exibir as primeiras linhas do dataframe
        

        return dsales, dprices, dcalendarios_eventos

    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado: {e}")
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")

In [13]:
# Carregando so datasets 

In [14]:
%%time
dsales, dprices, dcalendarios_eventos=read_dataset()

"item_sales.csv" imported successfully!
"item_prices.csv" imported successfully!
"daily_calendar_with_events.csv" imported successfully!
Wall time: 4.77 s


## OBS: este EDA está send trabalhado para gerar  a granularidaded de  item / semana / preco/ qtdvenda / ... Ou seja, estamos trabalhandmo todas os dataframe com a ganularidade semana antes do merge  

In [15]:
# Preparando o dataset de calendarios_eventos  paea poder realizar o Merge com o dataset de dprices 

In [16]:
dcalendarios_eventos

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN
...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,NaN
1909,2016-04-21,Thursday,6,d_1910,NaN
1910,2016-04-22,Friday,7,d_1911,NaN
1911,2016-04-23,Saturday,1,d_1912,NaN


In [17]:
dcalendarios_eventos.columns

Index(['date', 'weekday', 'weekday_int', 'd', 'event'], dtype='object')

In [18]:
# validando so nulos 

In [19]:
dcalendarios_eventos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [20]:
dcalendarios_eventos.isnull().sum()

date              0
weekday           0
weekday_int       0
d                 0
event          1887
dtype: int64

In [21]:
## preenchendo as linhas nulas de evento "Sem Inoformação"

In [22]:
# Preencher os valores nulos na coluna 'yearweek' com 190000
dcalendarios_eventos['event'] = dcalendarios_eventos['event'].fillna('Sem Inoformação')

In [23]:
dcalendarios_eventos.isnull().sum()

date           0
weekday        0
weekday_int    0
d              0
event          0
dtype: int64

In [24]:
dcalendarios_eventos

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,Sem Inoformação
1,2011-01-30,Sunday,2,d_2,Sem Inoformação
2,2011-01-31,Monday,3,d_3,Sem Inoformação
3,2011-02-01,Tuesday,4,d_4,Sem Inoformação
4,2011-02-02,Wednesday,5,d_5,Sem Inoformação
...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,Sem Inoformação
1909,2016-04-21,Thursday,6,d_1910,Sem Inoformação
1910,2016-04-22,Friday,7,d_1911,Sem Inoformação
1911,2016-04-23,Saturday,1,d_1912,Sem Inoformação


In [25]:
print(dcalendarios_eventos['date'].dtypes)

object


In [26]:
# criando das colunas semana, anosemana, a partir do campo date 

In [27]:
dcalendarios_eventos['date'] = pd.to_datetime(dcalendarios_eventos['date'], errors='coerce')

In [28]:
dcalendarios_eventos = dcalendarios_eventos.reset_index()

In [29]:
# Agora você pode usar isocalendar() para obter o número da semana
dcalendarios_eventos['weeknumber'] = dcalendarios_eventos['date'].dt.isocalendar().week

In [30]:
dcalendarios_eventos.columns

Index(['index', 'date', 'weekday', 'weekday_int', 'd', 'event', 'weeknumber'], dtype='object')

In [31]:
# criando a coluna ano 

In [32]:
# Agora você pode usar isocalendar() para obter o Ano
dcalendarios_eventos['year'] = dcalendarios_eventos['date'].dt.isocalendar().year

In [33]:
dcalendarios_eventos.columns

Index(['index', 'date', 'weekday', 'weekday_int', 'd', 'event', 'weeknumber',
       'year'],
      dtype='object')

In [34]:
# Criando da coluna anosemana

In [35]:
# Concatenar ano e número da semana no formato YYYYWW
dcalendarios_eventos['yearweek'] = dcalendarios_eventos['year'].astype(str) + dcalendarios_eventos['weeknumber'].astype(str).str.zfill(2)

In [36]:
dcalendarios_eventos.columns

Index(['index', 'date', 'weekday', 'weekday_int', 'd', 'event', 'weeknumber',
       'year', 'yearweek'],
      dtype='object')

In [37]:
dcalendarios_eventos.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
date,2011-01-29 00:00:00,2011-01-30 00:00:00,2011-01-31 00:00:00,2011-02-01 00:00:00,2011-02-02 00:00:00,2011-02-03 00:00:00,2011-02-04 00:00:00,2011-02-05 00:00:00,2011-02-06 00:00:00,2011-02-07 00:00:00,2011-02-08 00:00:00,2011-02-09 00:00:00,2011-02-10 00:00:00,2011-02-11 00:00:00,2011-02-12 00:00:00,2011-02-13 00:00:00,2011-02-14 00:00:00,2011-02-15 00:00:00,2011-02-16 00:00:00,2011-02-17 00:00:00
weekday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Monday,Tuesday,Wednesday,Thursday
weekday_int,1,2,3,4,5,6,7,1,2,3,4,5,6,7,1,2,3,4,5,6
d,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20
event,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,SuperBowl,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação,Sem Inoformação
weeknumber,4,4,5,5,5,5,5,5,5,6,6,6,6,6,6,6,7,7,7,7
year,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011,2011
yearweek,201104,201104,201105,201105,201105,201105,201105,201105,201105,201106,201106,201106,201106,201106,201106,201106,201107,201107,201107,201107


In [38]:
# passar a granularidade da tabela para semana , pois quero obter uma linha por evento semana 

In [39]:
 
# Agrupando por ano, número da semana e evento
dcalendarios_eventos_semanal = dcalendarios_eventos.groupby(['year','yearweek', 'event']).agg({
    'date': 'first',  # Você pode usar 'first', 'min', 'max' dependendo do que quer na coluna de data
    #'year': 'first',  # Você pode usar 'first', 'min', 'max' dependendo do que quer na coluna de data
    'weekday': 'first',  # Primeiro dia da semana correspondente ao evento
    'weekday_int': 'first'  # Ou outro critério que faça sentido para seu caso 
}).reset_index()

# Exibir o DataFrame resultante
print(dcalendarios_eventos_semanal.head())

   year yearweek            event       date   weekday  weekday_int
0  2011   201104  Sem Inoformação 2011-01-29  Saturday            1
1  2011   201105  Sem Inoformação 2011-01-31    Monday            3
2  2011   201105        SuperBowl 2011-02-06    Sunday            2
3  2011   201106  Sem Inoformação 2011-02-07    Monday            3
4  2011   201107  Sem Inoformação 2011-02-14    Monday            3


In [40]:
dcalendarios_eventos_semanal

,year,yearweek,event,date,weekday,weekday_int
0,2011,201104,Sem Inoformação,2011-01-29,Saturday,1
1,2011,201105,Sem Inoformação,2011-01-31,Monday,3
2,2011,201105,SuperBowl,2011-02-06,Sunday,2
3,2011,201106,Sem Inoformação,2011-02-07,Monday,3
4,2011,201107,Sem Inoformação,2011-02-14,Monday,3
5,2011,201108,Sem Inoformação,2011-02-21,Monday,3
6,2011,201109,Sem Inoformação,2011-02-28,Monday,3
7,2011,201110,Sem Inoformação,2011-03-07,Monday,3
8,2011,201111,Sem Inoformação,2011-03-14,Monday,3
9,2011,201112,Sem Inoformação,2011-03-21,Monday,3


In [41]:
total_linhas = dcalendarios_eventos_semanal.shape[0]
print(total_linhas)

300


In [42]:
dcalendarios_eventos_semanal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   year         300 non-null    int64         
 1   yearweek     300 non-null    object        
 2   event        300 non-null    object        
 3   date         300 non-null    datetime64[ns]
 4   weekday      300 non-null    object        
 5   weekday_int  300 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 14.2+ KB


In [43]:
dcalendarios_eventos_semanal.columns

Index(['year', 'yearweek', 'event', 'date', 'weekday', 'weekday_int'], dtype='object')

In [44]:
# realizando a ordenação do Datafrem para poder realizar o merge 

In [45]:
# Ordenando 
dcalendarios_eventos_semanal_sorted = dcalendarios_eventos_semanal.sort_values(by=['yearweek', 'date', 'year', 'weekday', 'weekday_int',  'event'], ascending=True)

# Exibindo o resultado ordenado
dcalendarios_eventos_semanal_sorted.head()

,year,yearweek,event,date,weekday,weekday_int
0,2011,201104,Sem Inoformação,2011-01-29,Saturday,1
1,2011,201105,Sem Inoformação,2011-01-31,Monday,3
2,2011,201105,SuperBowl,2011-02-06,Sunday,2
3,2011,201106,Sem Inoformação,2011-02-07,Monday,3
4,2011,201107,Sem Inoformação,2011-02-14,Monday,3


In [46]:
#Distinct 
valores_unicos_ordenados_evento = sorted(dcalendarios_eventos_semanal_sorted['yearweek'].unique())
print(valores_unicos_ordenados_evento)

['201104', '201105', '201106', '201107', '201108', '201109', '201110', '201111', '201112', '201113', '201114', '201115', '201116', '201117', '201118', '201119', '201120', '201121', '201122', '201123', '201124', '201125', '201126', '201127', '201128', '201129', '201130', '201131', '201132', '201133', '201134', '201135', '201136', '201137', '201138', '201139', '201140', '201141', '201142', '201143', '201144', '201145', '201146', '201147', '201148', '201149', '201150', '201151', '201152', '201201', '201202', '201203', '201204', '201205', '201206', '201207', '201208', '201209', '201210', '201211', '201212', '201213', '201214', '201215', '201216', '201217', '201218', '201219', '201220', '201221', '201222', '201223', '201224', '201225', '201226', '201227', '201228', '201229', '201230', '201231', '201232', '201233', '201234', '201235', '201236', '201237', '201238', '201239', '201240', '201241', '201242', '201243', '201244', '201245', '201246', '201247', '201248', '201249', '201250', '201251',

In [47]:
# Obtendo a menor data
menor_dcalendarios_eventos_semanal_sorted = dcalendarios_eventos_semanal_sorted['yearweek'].min()

# Obtendo a maior data
maior_dcalendarios_eventos_semanal_sorted = dcalendarios_eventos_semanal_sorted['yearweek'].max()

# Exibindo os resultados
print(f'Menor yearweek: {menor_dcalendarios_eventos_semanal_sorted}')
print(f'Maior yearweek: {maior_dcalendarios_eventos_semanal_sorted}')

Menor yearweek: 201104
Maior yearweek: 201616


In [48]:
dcalendarios_eventos_semanal_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   year         300 non-null    int64         
 1   yearweek     300 non-null    object        
 2   event        300 non-null    object        
 3   date         300 non-null    datetime64[ns]
 4   weekday      300 non-null    object        
 5   weekday_int  300 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 16.4+ KB


In [49]:
total_linhas_semanal = dcalendarios_eventos_semanal_sorted.shape[0]
print(total_linhas_semanal)

300


In [50]:
  	 	 	 	
# Encontrar as duplicatas
duplicados = dcalendarios_eventos_semanal_sorted.duplicated(subset=['year', 'yearweek', 'event', 'weekday','weekday_int'], keep=False)

# Filtrar o DataFrame original para mostrar apenas as linhas duplicadas
duplicados_dcalendarios_eventos_semanal_sorted = dcalendarios_eventos_semanal_sorted[duplicados]

# Exibir os itens duplicados
print(duplicados_dcalendarios_eventos_semanal_sorted)


Empty DataFrame
Columns: [year, yearweek, event, date, weekday, weekday_int]
Index: []


In [51]:
# convertendo weekday_int  300 non-null    int64  para int 32
dcalendarios_eventos_semanal_sorted[ 'weekday_int'] = dcalendarios_eventos_semanal_sorted[ 'weekday_int'].astype('int32')


In [52]:
# convertendo weekday_int  300 non-null    int64  para int 32
dcalendarios_eventos_semanal_sorted[ 'year'] = dcalendarios_eventos_semanal_sorted[ 'year'].astype('int32')

In [53]:
# convertendo yearweek  para  garantindo
dcalendarios_eventos_semanal_sorted[ 'yearweek'] = dcalendarios_eventos_semanal_sorted[ 'yearweek'].astype(int)

In [54]:
dcalendarios_eventos_semanal_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   year         300 non-null    int32         
 1   yearweek     300 non-null    int32         
 2   event        300 non-null    object        
 3   date         300 non-null    datetime64[ns]
 4   weekday      300 non-null    object        
 5   weekday_int  300 non-null    int32         
dtypes: datetime64[ns](1), int32(3), object(2)
memory usage: 12.9+ KB


In [55]:
percentual_total_nulos_evento_semanal_ordernado = dcalendarios_eventos_semanal_sorted.isnull().sum().sum() / dcalendarios_eventos_semanal_sorted.size * 100
print(f'Percentual total de nulos: {percentual_total_nulos_evento_semanal_ordernado:.2f}%')

Percentual total de nulos: 0.00%


In [56]:
# Verificando se o resultado é um DataFrame
print(dcalendarios_eventos_semanal_sorted)
print("Tipo:", type(dcalendarios_eventos_semanal_sorted))  # Deve ser um DataFrame

     year  yearweek            event       date    weekday  weekday_int
0    2011    201104  Sem Inoformação 2011-01-29   Saturday            1
1    2011    201105  Sem Inoformação 2011-01-31     Monday            3
2    2011    201105        SuperBowl 2011-02-06     Sunday            2
3    2011    201106  Sem Inoformação 2011-02-07     Monday            3
4    2011    201107  Sem Inoformação 2011-02-14     Monday            3
5    2011    201108  Sem Inoformação 2011-02-21     Monday            3
6    2011    201109  Sem Inoformação 2011-02-28     Monday            3
7    2011    201110  Sem Inoformação 2011-03-07     Monday            3
8    2011    201111  Sem Inoformação 2011-03-14     Monday            3
9    2011    201112  Sem Inoformação 2011-03-21     Monday            3
10   2011    201113  Sem Inoformação 2011-03-28     Monday            3
11   2011    201114  Sem Inoformação 2011-04-04     Monday            3
12   2011    201115  Sem Inoformação 2011-04-11     Monday      

In [57]:
# Tratando  o Datasdet de Precos --  Este dataset já está na granularidade Item, Precos e Semana 

In [58]:
dprices

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,328.00",12.74
1,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,329.00",12.74
2,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,330.00",10.99
3,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,331.00",10.99
4,ACCESORIES_1_001,ACCESORIES,NYC_1,"201,332.00",10.99
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.20


In [59]:
dprices.columns

Index(['item', 'category', 'store_code', 'yearweek', 'sell_price'], dtype='object')

In [60]:
dprices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [61]:
total_linhas_prices = dprices.shape[0]
print(total_linhas_prices)


6965706


In [62]:
## Verificando os valores nulos 

In [63]:
dprices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [64]:
percentual_total_nulos_prices = dprices.isnull().sum().sum() / dprices.size * 100
print(f'Percentual total de nulos: {percentual_total_nulos_prices:.2f}%')

Percentual total de nulos: 0.70%


In [65]:
# Encontrar as duplicatas  	 	 	 	
duplicados_dprices = dprices.duplicated(subset=['item', 'category', 'store_code', 'yearweek','sell_price'], keep=False)

# Filtrar o DataFrame original para mostrar apenas as linhas duplicadas
duplicados_prices = dprices[duplicados_dprices]

# Exibir os itens duplicados
print(duplicados_prices)

                      item     category store_code  yearweek  sell_price
149       ACCESORIES_1_001   ACCESORIES      NYC_1       NaN       11.15
150       ACCESORIES_1_001   ACCESORIES      NYC_1       NaN       11.15
151       ACCESORIES_1_001   ACCESORIES      NYC_1       NaN       11.15
152       ACCESORIES_1_001   ACCESORIES      NYC_1       NaN       11.15
153       ACCESORIES_1_001   ACCESORIES      NYC_1       NaN       11.15
...                    ...          ...        ...       ...         ...
6965701  SUPERMARKET_3_827  SUPERMARKET      PHI_3       NaN        1.20
6965702  SUPERMARKET_3_827  SUPERMARKET      PHI_3       NaN        1.20
6965703  SUPERMARKET_3_827  SUPERMARKET      PHI_3       NaN        1.20
6965704  SUPERMARKET_3_827  SUPERMARKET      PHI_3       NaN        1.20
6965705  SUPERMARKET_3_827  SUPERMARKET      PHI_3       NaN        1.20

[243495 rows x 5 columns]


In [66]:
# Essas linha por não terem o  yearweek , aparecem com sendo linhas duplicadas e com isso podendo será retiradas pos não teremos como utiliza-las nas analises

In [67]:
# Excluir duplicatas diretamente no DataFrame original com loc
dprices = dprices.loc[~dprices.duplicated(subset=['item', 'category', 'store_code', 'yearweek', 'sell_price'], keep='first')]

# Verificar o DataFrame após a exclusão das duplicatas
print(dprices)


                      item     category store_code   yearweek  sell_price
0         ACCESORIES_1_001   ACCESORIES      NYC_1 201,328.00       12.74
1         ACCESORIES_1_001   ACCESORIES      NYC_1 201,329.00       12.74
2         ACCESORIES_1_001   ACCESORIES      NYC_1 201,330.00       10.99
3         ACCESORIES_1_001   ACCESORIES      NYC_1 201,331.00       10.99
4         ACCESORIES_1_001   ACCESORIES      NYC_1 201,332.00       10.99
...                    ...          ...        ...        ...         ...
6965694  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,614.00        1.20
6965695  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,615.00        1.20
6965696  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,616.00        1.20
6965697  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,617.00        1.20
6965698  SUPERMARKET_3_827  SUPERMARKET      PHI_3        NaN        1.20

[6753586 rows x 5 columns]


In [68]:
total_linhas_dprices_sem_duplicatas = dprices.shape[0]
print(total_linhas_dprices_sem_duplicatas)

6753586


In [69]:
# verificando os nulos 

In [70]:
dprices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6753586 entries, 0 to 6965698
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 309.2+ MB


In [71]:
dprices.isnull().sum()

item              0
category          0
store_code        0
yearweek      31800
sell_price        0
dtype: int64

In [72]:
# tratano os nulos 

In [73]:
# Criar uma cópia independente do DataFrame original para evitar o "SettingWithCopyWarning"
dprices_copy = dprices.copy()

# Preencher os valores nulos na coluna 'yearweek' com 190000
dprices_copy.loc[:, 'yearweek'] = dprices_copy['yearweek'].fillna(190000)

# Exibir o DataFrame modificado
print(dprices_copy)



                      item     category store_code   yearweek  sell_price
0         ACCESORIES_1_001   ACCESORIES      NYC_1 201,328.00       12.74
1         ACCESORIES_1_001   ACCESORIES      NYC_1 201,329.00       12.74
2         ACCESORIES_1_001   ACCESORIES      NYC_1 201,330.00       10.99
3         ACCESORIES_1_001   ACCESORIES      NYC_1 201,331.00       10.99
4         ACCESORIES_1_001   ACCESORIES      NYC_1 201,332.00       10.99
...                    ...          ...        ...        ...         ...
6965694  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,614.00        1.20
6965695  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,615.00        1.20
6965696  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,616.00        1.20
6965697  SUPERMARKET_3_827  SUPERMARKET      PHI_3 201,617.00        1.20
6965698  SUPERMARKET_3_827  SUPERMARKET      PHI_3 190,000.00        1.20

[6753586 rows x 5 columns]


In [74]:
dprices_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6753586 entries, 0 to 6965698
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 309.2+ MB


In [75]:
# Tratando as tipos das colunnas

In [76]:
# Transformar a coluna 'yearweek' em int
dprices_copy['yearweek'] = dprices_copy['yearweek'].astype(int)

In [77]:
dprices_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6753586 entries, 0 to 6965698
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    int32  
 4   sell_price  float64
dtypes: float64(1), int32(1), object(3)
memory usage: 283.4+ MB


In [78]:
# Transformar a coluna 'sell_price ' em float16
dprices_copy['sell_price'] = dprices_copy['sell_price'].astype('float16')

In [79]:
dprices_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6753586 entries, 0 to 6965698
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    int32  
 4   sell_price  float16
dtypes: float16(1), int32(1), object(3)
memory usage: 244.7+ MB


In [80]:
percentual_total_nulos = dprices_copy.isnull().sum().sum() / dprices_copy.size * 100
print(f'Percentual total de nulos: {percentual_total_nulos:.2f}%')

Percentual total de nulos: 0.00%


In [81]:
total_linhas_prices = dprices_copy.shape[0]
print(total_linhas_prices)

6753586


In [82]:
# Verificando os tipos da do campo yearweek  nos  Dataframe

In [83]:
tipo_coluna_dprices = dprices_copy['yearweek'].dtype
print(tipo_coluna_dprices)


int32


In [84]:
tipo_coluna_events = dcalendarios_eventos_semanal_sorted['yearweek'].dtype
print(tipo_coluna_events)


int32


In [85]:
## converter o tipo do campo yearweek  object po  para poder realizar o merge com o dataframe calendario_evento

In [86]:
dprices_copy['yearweek'] = dprices_copy['yearweek'].astype(str)
#dcalendarios_eventos_semanal_sorted['yearweek'] = dcalendarios_eventos_semanal_sorted['yearweek'].astype(str)

In [87]:
tipo_coluna_dprices = dprices_copy['yearweek'].dtype
print(tipo_coluna_dprices)

object


In [88]:
# Atribuir a cópia modificada de volta ao DataFrame original
dprices = dprices_copy

In [89]:
dprices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6753586 entries, 0 to 6965698
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    object 
 4   sell_price  float16
dtypes: float16(1), object(4)
memory usage: 270.5+ MB


In [90]:
print(dprices['yearweek'].unique())

['201328' '201329' '201330' '201331' '201332' '201333' '201334' '201335'
 '201336' '201337' '201338' '201339' '201340' '201341' '201342' '201343'
 '201344' '201345' '201346' '201347' '201348' '201349' '201350' '201351'
 '201352' '201400' '201401' '201402' '201403' '201404' '201405' '201406'
 '201407' '201408' '201409' '201410' '201411' '201412' '201413' '201414'
 '201415' '201416' '201417' '201418' '201419' '201420' '201421' '201422'
 '201423' '201424' '201425' '201426' '201427' '201428' '201429' '201430'
 '201431' '201432' '201433' '201434' '201435' '201436' '201437' '201438'
 '201439' '201440' '201441' '201442' '201443' '201444' '201445' '201446'
 '201447' '201448' '201449' '201450' '201451' '201452' '201500' '201501'
 '201502' '201503' '201504' '201505' '201506' '201507' '201508' '201509'
 '201510' '201511' '201512' '201513' '201514' '201515' '201516' '201517'
 '201518' '201519' '201520' '201521' '201522' '201523' '201524' '201525'
 '201526' '201527' '201528' '201529' '201530' '2015

In [91]:
dprices.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
item,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001,ACCESORIES_1_001
category,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES,ACCESORIES
store_code,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1,NYC_1
yearweek,201328,201329,201330,201331,201332,201333,201334,201335,201336,201337,201338,201339,201340,201341,201342,201343,201344,201345,201346,201347
sell_price,12.74,12.74,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98,10.98


In [92]:
# ordemando o Dataset para o merge 
dprices_sorted = dprices.sort_values(by=['item', 'category', 'store_code', 'yearweek'], ascending=True)

# Exibindo o resultado ordenado
dprices_sorted.head()

,item,category,store_code,yearweek,sell_price
2758184,ACCESORIES_1_001,ACCESORIES,BOS_1,190000,10.98
2758035,ACCESORIES_1_001,ACCESORIES,BOS_1,201328,12.74
2758036,ACCESORIES_1_001,ACCESORIES,BOS_1,201329,10.98
2758037,ACCESORIES_1_001,ACCESORIES,BOS_1,201330,10.98
2758038,ACCESORIES_1_001,ACCESORIES,BOS_1,201331,10.98


In [93]:
# Obtendo a menor data
menor_dprices_sorted = dprices_sorted['yearweek'].min()

# Obtendo a maior data
maior_dprices_sorted = dprices_sorted['yearweek'].max()

# Exibindo os resultados
print(f'Menor data: {menor_dprices_sorted}')
print(f'Maior data: {maior_dprices_sorted}')

Menor data: 190000
Maior data: 201617


In [94]:
# Encontrar as duplicatas  	 	 	 	
duplicados_dprices_sorted = dprices_sorted.duplicated(subset=['item', 'category', 'store_code', 'yearweek','sell_price'], keep=False)

# Filtrar o DataFrame original para mostrar apenas as linhas duplicadas
duplicados_duplicados_dprices_sorted = dprices_sorted[duplicados_dprices_sorted]

# Exibir os itens duplicados
print(duplicados_duplicados_dprices_sorted)


Empty DataFrame
Columns: [item, category, store_code, yearweek, sell_price]
Index: []


In [95]:
# Relaizando o merge com outer para pode fazer o full dos dois Dataframe ordenadaos 

In [96]:
print("Antes do merge:", type(dprices_sorted), type(dcalendarios_eventos_semanal_sorted))

Antes do merge: <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [97]:
tipo_coluna_events = dcalendarios_eventos_semanal_sorted['yearweek'].dtype
print(tipo_coluna_events)

int32


In [98]:
tipo_coluna_prices = dprices_sorted['yearweek'].dtype
print(tipo_coluna_prices)

object


In [99]:
dcalendarios_eventos_semanal_sorted['yearweek'] = dcalendarios_eventos_semanal_sorted['yearweek'].astype(str)

In [100]:
tipo_coluna_dcalendarios_eventos_semanal_sorted = dcalendarios_eventos_semanal_sorted['yearweek'].dtype
print(tipo_coluna_dcalendarios_eventos_semanal_sorted)

object


In [101]:

# Fazendo o merge (Outer JOIN)
df_merged_prices_cevents = pd.merge(dprices_sorted, dcalendarios_eventos_semanal_sorted, on='yearweek', how='outer')

print(df_merged_prices_cevents)


                      item     category store_code yearweek  sell_price  \
0         ACCESORIES_1_001   ACCESORIES      BOS_1   190000       10.98   
1         ACCESORIES_1_001   ACCESORIES      BOS_2   190000       10.98   
2         ACCESORIES_1_001   ACCESORIES      BOS_3   190000       10.98   
3         ACCESORIES_1_001   ACCESORIES      NYC_1   190000       11.15   
4         ACCESORIES_1_001   ACCESORIES      NYC_2   190000       11.15   
...                    ...          ...        ...      ...         ...   
7358222  SUPERMARKET_3_825  SUPERMARKET      PHI_2   201109        4.80   
7358223  SUPERMARKET_3_825  SUPERMARKET      PHI_3   201109        4.80   
7358224                NaN          NaN        NaN   201104         NaN   
7358225                NaN          NaN        NaN   201553         NaN   
7358226                NaN          NaN        NaN   201553         NaN   

            year            event       date   weekday  weekday_int  
0            NaN             

In [102]:
# validando o nuumero de linhas pois em que aumentar  

In [103]:
total_linhas_merge_prices_cevents = df_merged_prices_cevents.shape[0]
print(total_linhas_merge_prices_cevents)


7358227


In [104]:
df_merged_prices_cevents.columns

Index(['item', 'category', 'store_code', 'yearweek', 'sell_price', 'year',
       'event', 'date', 'weekday', 'weekday_int'],
      dtype='object')

In [105]:
# Salvando o DataFrame em um arquivo CSV
df_merged_prices_cevents.to_csv('data/data_dsmarket/df_merged_prices_cevents1.csv', index=False)
# Exibindo a confirmação
print("Arquivo CSV salvo com sucesso!")

Arquivo CSV salvo com sucesso!


In [106]:
df_merged_prices_cevents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7358227 entries, 0 to 7358226
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   item         object        
 1   category     object        
 2   store_code   object        
 3   yearweek     object        
 4   sell_price   float16       
 5   year         float64       
 6   event        object        
 7   date         datetime64[ns]
 8   weekday      object        
 9   weekday_int  float64       
dtypes: datetime64[ns](1), float16(1), float64(2), object(6)
memory usage: 575.4+ MB


In [107]:
print(type(df_merged_prices_cevents))  # Isso mostrará o tipo do objeto. 


<class 'pandas.core.frame.DataFrame'>


In [108]:
# verificando os nulos 

In [109]:
df_merged_prices_cevents.isnull().sum()

item                3
category            3
store_code          3
yearweek            0
sell_price          3
year           208769
event          208769
date           208769
weekday        208769
weekday_int    208769
dtype: int64

In [110]:
# gerando analise do total de  nulos no Data frame 

In [111]:
percentual_total_nulos__merged_prices_cevents = df_merged_prices_cevents.isnull().sum().sum() / df_merged_prices_cevents.size * 100
print(f'Percentual total de nulos: {percentual_total_nulos__merged_prices_cevents:.2f}%')


Percentual total de nulos: 1.42%


In [112]:
df_merged_prices_cevents['sell_price'] = df_merged_prices_cevents['sell_price'].fillna(0)


In [113]:
df_merged_prices_cevents.isnull().sum()

item                3
category            3
store_code          3
yearweek            0
sell_price          0
year           208769
event          208769
date           208769
weekday        208769
weekday_int    208769
dtype: int64

In [114]:

# mostrando os campos nulos para os campo que tem eventos e não tem item 

In [115]:
# Exibindo linhas onde 'category' é nulo
df_nulos_category = df_merged_prices_cevents[df_merged_prices_cevents['category'].isna()]

print(df_nulos_category)


        item category store_code yearweek  sell_price     year  \
7358224  NaN      NaN        NaN   201104        0.00 2,011.00   
7358225  NaN      NaN        NaN   201553        0.00 2,015.00   
7358226  NaN      NaN        NaN   201553        0.00 2,015.00   

                   event       date   weekday  weekday_int  
7358224  Sem Inoformação 2011-01-29  Saturday         1.00  
7358225  Sem Inoformação 2015-12-28    Monday         3.00  
7358226          NewYear 2016-01-01    Friday         7.00  


In [116]:
# Preenchendo valores nulos na coluna 'category' com 'Sem Evento'
df_merged_prices_cevents['category'] = df_merged_prices_cevents['category'].fillna('Sem Categoria Evento')
                        

In [117]:
df_merged_prices_cevents.isnull().sum()

item                3
category            0
store_code          3
yearweek            0
sell_price          0
year           208769
event          208769
date           208769
weekday        208769
weekday_int    208769
dtype: int64

In [118]:
# Preenchendo valores nulos na coluna 'item' com 'Sem Evento'
df_merged_prices_cevents['item'] = df_merged_prices_cevents['item'].fillna('Item Sem Evento')

In [119]:
# Preenchendo valores nulos na coluna 'item' com 'S/E'
df_merged_prices_cevents['store_code'] = df_merged_prices_cevents['store_code'].fillna('S/CODE EVENT')

In [120]:
df_merged_prices_cevents.isnull().sum()

item                0
category            0
store_code          0
yearweek            0
sell_price          0
year           208769
event          208769
date           208769
weekday        208769
weekday_int    208769
dtype: int64

In [121]:
# Exibindo linhas onde 'category' é nulo
df_nulos_item = df_merged_prices_cevents[df_merged_prices_cevents['item'] == 'Item Sem Evento']

print(df_nulos_item)


                    item              category    store_code yearweek  \
7358224  Item Sem Evento  Sem Categoria Evento  S/CODE EVENT   201104   
7358225  Item Sem Evento  Sem Categoria Evento  S/CODE EVENT   201553   
7358226  Item Sem Evento  Sem Categoria Evento  S/CODE EVENT   201553   

         sell_price     year            event       date   weekday  \
7358224        0.00 2,011.00  Sem Inoformação 2011-01-29  Saturday   
7358225        0.00 2,015.00  Sem Inoformação 2015-12-28    Monday   
7358226        0.00 2,015.00          NewYear 2016-01-01    Friday   

         weekday_int  
7358224         1.00  
7358225         3.00  
7358226         7.00  


In [122]:
# validando os campos qie possui item e não tem evento 

In [123]:
# Exibindo linhas onde 'event' é nulo
df_nulos_event = df_merged_prices_cevents[df_merged_prices_cevents['event'].isna()]

print(df_nulos_event)

                      item     category store_code yearweek  sell_price  year  \
0         ACCESORIES_1_001   ACCESORIES      BOS_1   190000       10.98   NaN   
1         ACCESORIES_1_001   ACCESORIES      BOS_2   190000       10.98   NaN   
2         ACCESORIES_1_001   ACCESORIES      BOS_3   190000       10.98   NaN   
3         ACCESORIES_1_001   ACCESORIES      NYC_1   190000       11.15   NaN   
4         ACCESORIES_1_001   ACCESORIES      NYC_2   190000       11.15   NaN   
...                    ...          ...        ...      ...         ...   ...   
6587398  SUPERMARKET_3_825  SUPERMARKET      NYC_3   201300        4.78   NaN   
6587399  SUPERMARKET_3_825  SUPERMARKET      NYC_4   201300        5.26   NaN   
6587400  SUPERMARKET_3_825  SUPERMARKET      PHI_1   201300        5.26   NaN   
6587401  SUPERMARKET_3_825  SUPERMARKET      PHI_2   201300        5.26   NaN   
6587402  SUPERMARKET_3_825  SUPERMARKET      PHI_3   201300        5.26   NaN   

        event date weekday 

In [124]:
# Preenchendo valores nulos na coluna 'item' com 'Sem Evento'
df_merged_prices_cevents['event'] = df_merged_prices_cevents['event'].fillna('Evento Sem Item') 

In [125]:
df_merged_prices_cevents.isnull().sum()

item                0
category            0
store_code          0
yearweek            0
sell_price          0
year           208769
event               0
date           208769
weekday        208769
weekday_int    208769
dtype: int64

In [126]:
# Exibindo linhas onde 'year'  
df_nulos_year = df_merged_prices_cevents[df_merged_prices_cevents['event'] == 'Evento Sem Item']

print(df_nulos_event)

                      item     category store_code yearweek  sell_price  year  \
0         ACCESORIES_1_001   ACCESORIES      BOS_1   190000       10.98   NaN   
1         ACCESORIES_1_001   ACCESORIES      BOS_2   190000       10.98   NaN   
2         ACCESORIES_1_001   ACCESORIES      BOS_3   190000       10.98   NaN   
3         ACCESORIES_1_001   ACCESORIES      NYC_1   190000       11.15   NaN   
4         ACCESORIES_1_001   ACCESORIES      NYC_2   190000       11.15   NaN   
...                    ...          ...        ...      ...         ...   ...   
6587398  SUPERMARKET_3_825  SUPERMARKET      NYC_3   201300        4.78   NaN   
6587399  SUPERMARKET_3_825  SUPERMARKET      NYC_4   201300        5.26   NaN   
6587400  SUPERMARKET_3_825  SUPERMARKET      PHI_1   201300        5.26   NaN   
6587401  SUPERMARKET_3_825  SUPERMARKET      PHI_2   201300        5.26   NaN   
6587402  SUPERMARKET_3_825  SUPERMARKET      PHI_3   201300        5.26   NaN   

        event date weekday 

In [127]:
# Exibindo linhas onde 'year' é nulo
df_nulos_year = df_merged_prices_cevents[df_merged_prices_cevents['year'].isna()]

print(df_nulos_year)

                      item     category store_code yearweek  sell_price  year  \
0         ACCESORIES_1_001   ACCESORIES      BOS_1   190000       10.98   NaN   
1         ACCESORIES_1_001   ACCESORIES      BOS_2   190000       10.98   NaN   
2         ACCESORIES_1_001   ACCESORIES      BOS_3   190000       10.98   NaN   
3         ACCESORIES_1_001   ACCESORIES      NYC_1   190000       11.15   NaN   
4         ACCESORIES_1_001   ACCESORIES      NYC_2   190000       11.15   NaN   
...                    ...          ...        ...      ...         ...   ...   
6587398  SUPERMARKET_3_825  SUPERMARKET      NYC_3   201300        4.78   NaN   
6587399  SUPERMARKET_3_825  SUPERMARKET      NYC_4   201300        5.26   NaN   
6587400  SUPERMARKET_3_825  SUPERMARKET      PHI_1   201300        5.26   NaN   
6587401  SUPERMARKET_3_825  SUPERMARKET      PHI_2   201300        5.26   NaN   
6587402  SUPERMARKET_3_825  SUPERMARKET      PHI_3   201300        5.26   NaN   

                   event da

In [128]:
# Preenchendo valores nulos na coluna 'year' com 'com o ano do campo yearweek '
# Preenchendo valores nulos na coluna 'year' com o ano extraído do campo 'yearweek'
df_merged_prices_cevents['year'] = df_merged_prices_cevents['year'].fillna(
    df_merged_prices_cevents['yearweek'].astype(str).str[:4]
)


In [129]:
df_merged_prices_cevents.isnull().sum()

item                0
category            0
store_code          0
yearweek            0
sell_price          0
year                0
event               0
date           208769
weekday        208769
weekday_int    208769
dtype: int64

In [130]:
 
# Preenchendo valores nulos na coluna 'weekday_int' com o ano extraído do campo '0'
df_merged_prices_cevents['weekday_int'] = df_merged_prices_cevents['weekday_int'].fillna(0)

In [131]:
df_merged_prices_cevents.isnull().sum()

item                0
category            0
store_code          0
yearweek            0
sell_price          0
year                0
event               0
date           208769
weekday        208769
weekday_int         0
dtype: int64

In [132]:
 
# Preenchendo valores nulos na coluna 'weekday_int' com o ano extraído do campo '0'
df_merged_prices_cevents['weekday'] = df_merged_prices_cevents['weekday'].fillna(0)

In [133]:
df_merged_prices_cevents.isnull().sum()

item                0
category            0
store_code          0
yearweek            0
sell_price          0
year                0
event               0
date           208769
weekday             0
weekday_int         0
dtype: int64

In [134]:
 
# Preenchendo valores nulos na coluna 'date' com '01/01/1900'
df_merged_prices_cevents['date'] = df_merged_prices_cevents['date'].fillna('01/01/1900')


In [135]:
 # Deve retornar 0 se todos os valores nulos foram preenchidos
print(df_merged_prices_cevents['date'].isnull().sum()) 


0


In [136]:
# Exibindo linhas onde 'category' é nulo
df_nulos_date = df_merged_prices_cevents[df_merged_prices_cevents['date'] ==  '01/01/1900']

print(df_nulos_date)

                      item     category store_code yearweek  sell_price  year  \
0         ACCESORIES_1_001   ACCESORIES      BOS_1   190000       10.98  1900   
1         ACCESORIES_1_001   ACCESORIES      BOS_2   190000       10.98  1900   
2         ACCESORIES_1_001   ACCESORIES      BOS_3   190000       10.98  1900   
3         ACCESORIES_1_001   ACCESORIES      NYC_1   190000       11.15  1900   
4         ACCESORIES_1_001   ACCESORIES      NYC_2   190000       11.15  1900   
...                    ...          ...        ...      ...         ...   ...   
6587398  SUPERMARKET_3_825  SUPERMARKET      NYC_3   201300        4.78  2013   
6587399  SUPERMARKET_3_825  SUPERMARKET      NYC_4   201300        5.26  2013   
6587400  SUPERMARKET_3_825  SUPERMARKET      PHI_1   201300        5.26  2013   
6587401  SUPERMARKET_3_825  SUPERMARKET      PHI_2   201300        5.26  2013   
6587402  SUPERMARKET_3_825  SUPERMARKET      PHI_3   201300        5.26  2013   

                   event   

In [137]:
print(df_merged_prices_cevents.isnull().sum())  # Deve retornar 0 se todos os valores nulos foram preenchidos


item           0
category       0
store_code     0
yearweek       0
sell_price     0
year           0
event          0
date           0
weekday        0
weekday_int    0
dtype: int64


In [138]:
# Ordernando pelo Item o dataframe , preparandopara o  proximo merge 

In [139]:
# Ordenando o DataFrame 'df_merged_prices_cevents' pela coluna 'item' em ordem ascendente
df_merged_prices_cevents_sort= df_merged_prices_cevents.sort_values(by='item', ascending= True)

# Exibindo o DataFrame ordenado
print(df_merged_prices_cevents_sort)

                      item     category store_code yearweek  sell_price  \
0         ACCESORIES_1_001   ACCESORIES      BOS_1   190000       10.98   
4008115   ACCESORIES_1_001   ACCESORIES      BOS_2   201550       10.98   
4008114   ACCESORIES_1_001   ACCESORIES      BOS_1   201550       10.98   
3977657   ACCESORIES_1_001   ACCESORIES      PHI_3   201549       10.98   
3977656   ACCESORIES_1_001   ACCESORIES      PHI_2   201549       11.15   
...                    ...          ...        ...      ...         ...   
3187210  SUPERMARKET_3_827  SUPERMARKET      NYC_3   201524        1.20   
3187211  SUPERMARKET_3_827  SUPERMARKET      NYC_4   201524        1.20   
3187212  SUPERMARKET_3_827  SUPERMARKET      PHI_1   201524        1.20   
4556768  SUPERMARKET_3_827  SUPERMARKET      NYC_4   201612        1.20   
1449053  SUPERMARKET_3_827  SUPERMARKET      BOS_2   201423        1.20   

            year            event       date weekday  weekday_int  
0           1900  Evento Sem It

In [140]:
# Preparano a tabela de item qtdvendida

In [141]:
dsales

id               item     category     department  \
0       ACCESORIES_1_001_NYC_1   ACCESORIES_1_001   ACCESORIES   ACCESORIES_1   
1       ACCESORIES_1_002_NYC_1   ACCESORIES_1_002   ACCESORIES   ACCESORIES_1   
2       ACCESORIES_1_003_NYC_1   ACCESORIES_1_003   ACCESORIES   ACCESORIES_1   
3       ACCESORIES_1_004_NYC_1   ACCESORIES_1_004   ACCESORIES   ACCESORIES_1   
4       ACCESORIES_1_005_NYC_1   ACCESORIES_1_005   ACCESORIES   ACCESORIES_1   
...                        ...                ...          ...            ...   
30485  SUPERMARKET_3_823_PHI_3  SUPERMARKET_3_823  SUPERMARKET  SUPERMARKET_3   
30486  SUPERMARKET_3_824_PHI_3  SUPERMARKET_3_824  SUPERMARKET  SUPERMARKET_3   
30487  SUPERMARKET_3_825_PHI_3  SUPERMARKET_3_825  SUPERMARKET  SUPERMARKET_3   
30488  SUPERMARKET_3_826_PHI_3  SUPERMARKET_3_826  SUPERMARKET  SUPERMARKET_3   
30489  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET  SUPERMARKET_3   

                   store store_code        region  d_1  d_2  d_3  d_4  d_5  \
0      Greenwich_Village      NYC_1      New York    0    0    0    0    0   
1      Greenwich_Village      NYC_1      New York    0    0    0    0    0   
2      Greenwich_Village      NYC_1      New York    0    0    0    0    0   
3      Greenwich_Village      NYC_1      New York    0    0    0    0    0   
4      Greenwich_Village      NYC_1      New York    0    0    0    0    0   
...                  ...        ...           ...  ...  ...  ...  ...  ...   
30485      Queen_Village      PHI_3  Philadelphia    0    0    2    2    0   
30486      Queen_Village      PHI_3  Philadelphia    0    0    0    0    0   
30487      Queen_Village      PHI_3  Philadelphia    0    6    0    2    2   
30488      Queen_Village      PHI_3  Philadelphia    0    0    0    0    0   
30489      Queen_Village      PHI_3  Philadelphia    0    0    0    0    0   

       d_6  d_7  d_8  d_9  d_10  d_11  d_12  d_13  d_14  d_15  d_16  d_17  \
0        0    0    0    0     0     0     0     0     0     0     0     0   
1        0    0    0    0     0     0     0     0     0     0     0     0   
2        0    0    0    0     0     0     0     0     0     0     0     0   
3        0    0    0    0     0     0     0     0     0     0     0     0   
4        0    0    0    0     0     0     0     0     0     0     0     0   
...    ...  ...  ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485    3    1    4    1     0     0     3     4     4     0     0     1   
30486    5    0    1    1     3     1     1     0     4     2     0     1   
30487    4    1    8    5     2     7     5     3     5    20     8    10   
30488    0    0    0    0     0     0     0     0     0     0     0     0   
30489    0    0    0    0     0     0     0     0     0     0     0     0   

       d_18  d_19  d_20  d_21  d_22  d_23  d_24  d_25  d_26  d_27  d_28  d_29  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     1     1     7     7     3     6     3     3     7    12     4   
30486     2     1     1     0     0     0     0     3     1     1     1     2   
30487     3     3     4     7     2     3     5     6     3     4     1     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_30  d_31  d_32  d_33  d_34  d_35  d_36  d_37  d_38  d_39  d_40  d_41  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     

In [142]:
dsales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [143]:
dsales.columns

Index(['id', 'item', 'category', 'department', 'store', 'store_code', 'region',
       'd_1', 'd_2', 'd_3',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1920)

In [144]:
 # Verificar o total de linhas no DataFrame dsales_weekly_sort
total_linhas_dsales = len(dsales)
print(f"Total de linhas em dsales_weekly_sort: {total_linhas_dsales}")


Total de linhas em dsales_weekly_sort: 30490


In [145]:
# verificando se tem linhas duplicadas 

In [146]:

# Encontrar as duplicatas
duplicados_dsales = dsales.duplicated(subset=['item', 'category', 'department', 'store','region'], keep=False)

# Filtrar o DataFrame original para mostrar apenas as linhas duplicadas
duplicados_df_sales	= dsales[duplicados_dsales]

# Exibir os itens duplicados
print(duplicados_df_sales)


Empty DataFrame
Columns: [id, item, category, department, store, store_code, region, d_1, d_2, d_3, d_4, d_5, d_6, d_7, d_8, d_9, d_10, d_11, d_12, d_13, d_14, d_15, d_16, d_17, d_18, d_19, d_20, d_21, d_22, d_23, d_24, d_25, d_26, d_27, d_28, d_29, d_30, d_31, d_32, d_33, d_34, d_35, d_36, d_37, d_38, d_39, d_40, d_41, d_42, d_43, d_44, d_45, d_46, d_47, d_48, d_49, d_50, d_51, d_52, d_53, d_54, d_55, d_56, d_57, d_58, d_59, d_60, d_61, d_62, d_63, d_64, d_65, d_66, d_67, d_68, d_69, d_70, d_71, d_72, d_73, d_74, d_75, d_76, d_77, d_78, d_79, d_80, d_81, d_82, d_83, d_84, d_85, d_86, d_87, d_88, d_89, d_90, d_91, d_92, d_93, ...]
Index: []


In [147]:
# nao temos duplicados no arquivo

In [148]:
# primeiro tratamento e  calculando a media por semana, para transformar a granularidade de dia para semana

In [149]:
# Cria a lista de colunas de dias
day_columns = [f'd_{i}' for i in range(1, 1914)]

# Calcula o número de semanas (cada grupo de 7 dias é uma semana)
n_weeks = len(day_columns) // 7

# Agrupa os dias em semanas e calcula a média
weekly_means = []

# Itera sobre as semanas
for i in range(n_weeks):
    start_day = i * 7 + 1  # Dia inicial da semana
    end_day = start_day + 6  # Dia final da semana
    
    # Cria uma lista com os dias da semana atual
    week_days = [f'd_{j}' for j in range(start_day, end_day + 1)]
    
    # Calcula a média para essa semana
    dsales[f'dweek_{i+1}'] = dsales[week_days].mean(axis=1)
    weekly_means.append(f'dweek_{i+1}')

# Agora, df tem colunas para cada semana com a média das vendas
# Podemos organizar os dados e remover as colunas de dias, se necessário:
dsales_weekly = dsales[['id', 'item', 'category', 'department', 'store', 'store_code', 'region'] + weekly_means]

# Exibe o resultado
dsales_weekly.head()

,id,item,category,department,store,store_code,region,dweek_1,dweek_2,dweek_3,dweek_4,dweek_5,dweek_6,dweek_7,dweek_8,dweek_9,dweek_10,dweek_11,dweek_12,dweek_13,dweek_14,dweek_15,dweek_16,dweek_17,dweek_18,dweek_19,dweek_20,dweek_21,dweek_22,dweek_23,dweek_24,dweek_25,dweek_26,dweek_27,dweek_28,dweek_29,dweek_30,dweek_31,dweek_32,dweek_33,dweek_34,dweek_35,dweek_36,dweek_37,dweek_38,dweek_39,dweek_40,dweek_41,dweek_42,dweek_43,dweek_44,dweek_45,dweek_46,dweek_47,dweek_48,dweek_49,dweek_50,dweek_51,dweek_52,dweek_53,dweek_54,dweek_55,dweek_56,dweek_57,dweek_58,dweek_59,dweek_60,dweek_61,dweek_62,dweek_63,dweek_64,dweek_65,dweek_66,dweek_67,dweek_68,dweek_69,dweek_70,dweek_71,dweek_72,dweek_73,dweek_74,dweek_75,dweek_76,dweek_77,dweek_78,dweek_79,dweek_80,dweek_81,dweek_82,dweek_83,dweek_84,dweek_85,dweek_86,dweek_87,dweek_88,dweek_89,dweek_90,dweek_91,dweek_92,dweek_93,dweek_94,dweek_95,dweek_96,dweek_97,dweek_98,dweek_99,dweek_100,dweek_101,dweek_102,dweek_103,dweek_104,dweek_105,dweek_106,dweek_107,dweek_108,dweek_109,dweek_110,dweek_111,dweek_112,dweek_113,dweek_114,dweek_115,dweek_116,dweek_117,dweek_118,dweek_119,dweek_120,dweek_121,dweek_122,dweek_123,dweek_124,dweek_125,dweek_126,dweek_127,dweek_128,dweek_129,dweek_130,dweek_131,dweek_132,dweek_133,dweek_134,dweek_135,dweek_136,dweek_137,dweek_138,dweek_139,dweek_140,dweek_141,dweek_142,dweek_143,dweek_144,dweek_145,dweek_146,dweek_147,dweek_148,dweek_149,dweek_150,dweek_151,dweek_152,dweek_153,dweek_154,dweek_155,dweek_156,dweek_157,dweek_158,dweek_159,dweek_160,dweek_161,dweek_162,dweek_163,dweek_164,dweek_165,dweek_166,dweek_167,dweek_168,dweek_169,dweek_170,dweek_171,dweek_172,dweek_173,dweek_174,dweek_175,dweek_176,dweek_177,dweek_178,dweek_179,dweek_180,dweek_181,dweek_182,dweek_183,dweek_184,dweek_185,dweek_186,dweek_187,dweek_188,dweek_189,dweek_190,dweek_191,dweek_192,dweek_193,dweek_194,dweek_195,dweek_196,dweek_197,dweek_198,dweek_199,dweek_200,dweek_201,dweek_202,dweek_203,dweek_204,dweek_205,dweek_206,dweek_207,dweek_208,dweek_209,dweek_210,dweek_211,dweek_212,dweek_213,dweek_214,dweek_215,dweek_216,dweek_217,dweek_218,dweek_219,dweek_220,dweek_221,dweek_222,dweek_223,dweek_224,dweek_225,dweek_226,dweek_227,dweek_228,dweek_229,dweek_230,dweek_231,dweek_232,dweek_233,dweek_234,dweek_235,dweek_236,dweek_237,dweek_238,dweek_239,dweek_240,dweek_241,dweek_242,dweek_243,dweek_244,dweek_245,dweek_246,dweek_247,dweek_248,dweek_249,dweek_250,dweek_251,dweek_252,dweek_253,dweek_254,dweek_255,dweek_256,dweek_257,dweek_258,dweek_259,dweek_260,dweek_261,dweek_262,dweek_263,dweek_264,dweek_265,dweek_266,dweek_267,dweek_268,dweek_269,dweek_270,dweek_271,dweek_272,dweek_273
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.14,0.00,0.29,0.29,0.86,0.14,0.29,0.43,0.43,0.57,0.43,0.57,0.43,0.29,0.43,0.57,0.86,1.00,0.14,0.71,0.86,0.86,1.00,0.43,0.29,0.43,0.43,0.29,0.29,0.29,0.29,0.86,0.43,0.29,0.14,0.57,0.57,0.00,0.57,0.29,0.43,0.29,0.14,0.14,0.71,0.29,0.29,0.43,0.57,0.57,1.14,0.29,0.71,0.29,0.43,0.57,0.14,1.00,0.14,0.86,0.86,0.71,0.29,0.71,1.00,0.29,0.29,0.57,0.43,0.71,0.71,0.86,0.71,0.43,0.86,0.57,1.14,0.43,0.86,0.43,0.71,0.57,1.00,0.43,1.00,0.57,0.71,0.57,1.14,0.86,1.00,0.43,0.71,0.57,0.43,0.00,0.00,0.00,0.00,0.29,0.57,0.57,0.57,0.86,0.86,0.71,0.29,1.00,0.71,0.71,0.86,0.86,0.71,0.71,0.14,1.00,0.57,0

In [150]:
total_linhas_dsales_weekly = dsales_weekly.shape[0]
print(total_linhas_dsales_weekly)

30490


In [151]:
# Ordenano o  campo 'item' dos dataframe para poder aplicar o merge 

In [152]:
# Ordenando o DataFrame 'dsales_semanal' pela coluna 'item' em ordem ascendente
dsales_weekly_sort = dsales_weekly.sort_values(by='item', ascending= True)

# Exibindo o DataFrame ordenado
print(dsales_weekly_sort)


                            id               item     category     department  \
0       ACCESORIES_1_001_NYC_1   ACCESORIES_1_001   ACCESORIES   ACCESORIES_1   
21343   ACCESORIES_1_001_PHI_1   ACCESORIES_1_001   ACCESORIES   ACCESORIES_1   
15245   ACCESORIES_1_001_BOS_2   ACCESORIES_1_001   ACCESORIES   ACCESORIES_1   
12196   ACCESORIES_1_001_BOS_1   ACCESORIES_1_001   ACCESORIES   ACCESORIES_1   
24392   ACCESORIES_1_001_PHI_2   ACCESORIES_1_001   ACCESORIES   ACCESORIES_1   
...                        ...                ...          ...            ...   
9146   SUPERMARKET_3_827_NYC_3  SUPERMARKET_3_827  SUPERMARKET  SUPERMARKET_3   
6097   SUPERMARKET_3_827_NYC_2  SUPERMARKET_3_827  SUPERMARKET  SUPERMARKET_3   
3048   SUPERMARKET_3_827_NYC_1  SUPERMARKET_3_827  SUPERMARKET  SUPERMARKET_3   
27440  SUPERMARKET_3_827_PHI_2  SUPERMARKET_3_827  SUPERMARKET  SUPERMARKET_3   
30489  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET  SUPERMARKET_3   

                   store st

In [153]:
# Aplicando o merge dos dataframe pela coluna itens 

In [154]:
total_linhas_dsales_weekly_sort = dsales_weekly_sort.shape[0]
print(total_linhas_dsales_weekly_sort)


30490


In [155]:
dsales_weekly_sort.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30490 entries, 0 to 30489
Columns: 280 entries, id to dweek_273
dtypes: float64(273), object(7)
memory usage: 65.4+ MB


In [156]:
dsales_weekly_sort.columns

Index(['id', 'item', 'category', 'department', 'store', 'store_code', 'region',
       'dweek_1', 'dweek_2', 'dweek_3',
       ...
       'dweek_264', 'dweek_265', 'dweek_266', 'dweek_267', 'dweek_268',
       'dweek_269', 'dweek_270', 'dweek_271', 'dweek_272', 'dweek_273'],
      dtype='object', length=280)

In [157]:

# Identificar as colunas do tipo float64 no DataFrame
colunas_float64 = dsales_weekly_sort.select_dtypes(include=['float64']).columns

# Converter as colunas de float64 para float16
dsales_weekly_sort[colunas_float64] = dsales_weekly_sort[colunas_float64].astype('float16')

# Verificar o resultado
print(dsales_weekly_sort.dtypes)



id             object
item           object
category       object
department     object
store          object
store_code     object
region         object
dweek_1       float16
dweek_2       float16
dweek_3       float16
dweek_4       float16
dweek_5       float16
dweek_6       float16
dweek_7       float16
dweek_8       float16
dweek_9       float16
dweek_10      float16
dweek_11      float16
dweek_12      float16
dweek_13      float16
dweek_14      float16
dweek_15      float16
dweek_16      float16
dweek_17      float16
dweek_18      float16
dweek_19      float16
dweek_20      float16
dweek_21      float16
dweek_22      float16
dweek_23      float16
dweek_24      float16
dweek_25      float16
dweek_26      float16
dweek_27      float16
dweek_28      float16
dweek_29      float16
dweek_30      float16
dweek_31      float16
dweek_32      float16
dweek_33      float16
dweek_34      float16
dweek_35      float16
dweek_36      float16
dweek_37      float16
dweek_38      float16
dweek_39  

In [158]:
#fazendo o melt 

In [159]:
import vaex
print(vaex.__version__)


{'vaex': '4.17.0', 'vaex-core': '4.17.1', 'vaex-viz': '0.5.4', 'vaex-hdf5': '0.14.1', 'vaex-server': '0.9.0', 'vaex-astro': '0.9.3', 'vaex-jupyter': '0.8.2', 'vaex-ml': '0.18.3'}


In [160]:
import dask.dataframe as dd
import vaex

In [161]:
import vaex

# Carregar o DataFrame como Vaex
dsales_weekly_sort_vx = vaex.from_pandas(dsales_weekly_sort)

# Converter colunas categóricas para strings (ou 'category' se suportado pelo Vaex)
for col in ['category', 'department', 'store', 'region', 'store_code']:
    dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('string')

# Converter colunas float64 para float32
float_columns = dsales_weekly_sort_vx.get_column_names()  # Obter nomes das colunas
for col in float_columns:
    if dsales_weekly_sort_vx[col].dtype == 'float64':
        dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('float32')  # Vaex não suporta float16

# Criar uma lista para armazenar os DataFrames temporários
melted = []
for week in range(1, 274):
    week_col = f'dweek_{week}'
    if week_col in dsales_weekly_sort_vx.get_column_names():  # Verificar se a coluna existe
        # Selecionar as colunas necessárias
        temp_df = dsales_weekly_sort_vx[['id', 'item', 'category', 'department', 'store', 'store_code', 'region', week_col]]
        
        # Renomear a coluna week_col para 'sales'
        temp_df['sales'] = temp_df[week_col]  # Atribuir os valores da coluna week_col à nova coluna 'sales'
        
        # Adicionar a temp_df à lista melted
        melted.append(temp_df[['id', 'item', 'category', 'department', 'store', 'store_code', 'region', 'sales']])  # Manter as colunas necessárias

# Concatenar todos os DataFrames em um único DataFrame Vaex
dsales_weekly_sort_melt_vx = vaex.concat(melted)

# Verificando o resultado
print(dsales_weekly_sort_melt_vx.head())





  #  id                      item                sales  region        store              category    department    store_code
  0  ACCESORIES_1_001_NYC_1  ACCESORIES_1_001        0  New York      Greenwich_Village  ACCESORIES  ACCESORIES_1  NYC_1
  1  ACCESORIES_1_001_PHI_1  ACCESORIES_1_001        0  Philadelphia  Midtown_Village    ACCESORIES  ACCESORIES_1  PHI_1
  2  ACCESORIES_1_001_BOS_2  ACCESORIES_1_001        0  Boston        Roxbury            ACCESORIES  ACCESORIES_1  BOS_2
  3  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001        0  Boston        South_End          ACCESORIES  ACCESORIES_1  BOS_1
  4  ACCESORIES_1_001_PHI_2  ACCESORIES_1_001        0  Philadelphia  Yorktown           ACCESORIES  ACCESORIES_1  PHI_2
  5  ACCESORIES_1_001_BOS_3  ACCESORIES_1_001        0  Boston        Back_Bay           ACCESORIES  ACCESORIES_1  BOS_3
  6  ACCESORIES_1_001_NYC_2  ACCESORIES_1_001        0  New York      Harlem             ACCESORIES  ACCESORIES_1  NYC_2
  7  ACCESORIES_1_001_NYC_4

In [162]:
dsales_weekly_sort_melt_vx.info()

In [163]:
 # Verificar o total de linhas no DataFrame dsales_weekly_sort
total_linhas_dsales_weekly_sort_melt_vx = len(dsales_weekly_sort_melt_vx)
print(f"Total de linhas em dsales_weekly_sort: {total_linhas_dsales_weekly_sort_melt_vx}")  #8323770

Total de linhas em dsales_weekly_sort: 8323770


In [164]:
import vaex

# Carregar o DataFrame como Vaex
dsales_weekly_sort_vx = vaex.from_pandas(dsales_weekly_sort)

# Converter colunas categóricas para string
categorical_cols = ['category', 'department', 'store', 'region', 'store_code']
for col in categorical_cols:
    dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('string')

# Converter colunas float64 para float32
float_columns = dsales_weekly_sort_vx.get_column_names()  # Obter nomes das colunas
for col in float_columns:
    if dsales_weekly_sort_vx[col].dtype == 'float64':
        dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('float32')  # Convertendo para float32

# Criar uma lista para armazenar os DataFrames temporários
melted = []
for week in range(1, 274):
    week_col = f'dweek_{week}'
    if week_col in dsales_weekly_sort_vx.get_column_names():  # Verificar se a coluna existe
        # Selecionar as colunas necessárias
        temp_df = dsales_weekly_sort_vx[['id', 'item', 'department', 'store', 'store_code', 'region', week_col]].copy()
        
        # Renomear a coluna week_col para 'sales' e adicionar uma nova coluna 'week'
        temp_df['sales'] = temp_df[week_col]  # Atribuir os valores da coluna week_col à nova coluna 'sales'
        temp_df['week'] = week_col  # Adicionar a nova coluna 'week' com o nome da semana
        
        # Manter apenas as colunas necessárias e adicionar à lista melted
        melted.append(temp_df[['id', 'item', 'department', 'store', 'store_code', 'region', 'week', 'sales']])

# Concatenar todos os DataFrames em um único DataFrame Vaex
dsales_weekly_sort_melt_vx = vaex.concat(melted)

# Ajustar tipos de dados finais para os menores tipos possíveis
dsales_weekly_sort_melt_vx['sales'] = dsales_weekly_sort_melt_vx['sales'].astype('float32')  # float32 como menor tipo

# Verificando os tipos de dados finais
dsales_weekly_sort_melt_vx['id'] = dsales_weekly_sort_melt_vx['id'].astype('string')  # String para ID
dsales_weekly_sort_melt_vx['item'] = dsales_weekly_sort_melt_vx['item'].astype('string')  # String para item

# Verificando a estrutura final
print(dsales_weekly_sort_melt_vx.dtypes)

# Verificando o resultado
print(dsales_weekly_sort_melt_vx.head())


id             string
item           string
week          float16
sales         float32
region         string
store          string
department     string
store_code     string
dtype: object
  #  id                      item                week    sales  region        store              department    store_code
  0  ACCESORIES_1_001_NYC_1  ACCESORIES_1_001       0        0  New York      Greenwich_Village  ACCESORIES_1  NYC_1
  1  ACCESORIES_1_001_PHI_1  ACCESORIES_1_001       0        0  Philadelphia  Midtown_Village    ACCESORIES_1  PHI_1
  2  ACCESORIES_1_001_BOS_2  ACCESORIES_1_001       0        0  Boston        Roxbury            ACCESORIES_1  BOS_2
  3  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001       0        0  Boston        South_End          ACCESORIES_1  BOS_1
  4  ACCESORIES_1_001_PHI_2  ACCESORIES_1_001       0        0  Philadelphia  Yorktown           ACCESORIES_1  PHI_2
  5  ACCESORIES_1_001_BOS_3  ACCESORIES_1_001       0        0  Boston        Back_Bay           ACCESO

In [165]:
import vaex

# Carregar o DataFrame como Vaex
dsales_weekly_sort_vx = vaex.from_pandas(dsales_weekly_sort)

# Converter colunas categóricas para string (já que Vaex não suporta 'category')
categorical_columns = ['category', 'department', 'store', 'region', 'store_code']
for col in categorical_columns:
    dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('string')

# Converter colunas float64 para float32
float_columns = dsales_weekly_sort_vx.column_names
for col in float_columns:
    if dsales_weekly_sort_vx[col].dtype == 'float64':
        dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('float32')  # Vaex não suporta float16

# Criar uma lista para armazenar os DataFrames temporários
melted = []
for week in range(1, 274):
    week_col = f'dweek_{week}'
    if week_col in dsales_weekly_sort_vx.column_names:  # Verificar se a coluna existe
        # Selecionar as colunas necessárias
        temp_df = dsales_weekly_sort_vx[['id', 'item', 'department', 'store', 'store_code', 'region', week_col]]
        
        # Renomear a coluna week_col para sales
        temp_df['sales'] = temp_df[week_col]  # Adiciona a coluna de vendas
        temp_df = temp_df.drop(columns=[week_col])  # Remove a coluna original

        # Adicionar a temp_df à lista melted
        melted.append(temp_df)

# Concatenar todos os DataFrames em um único DataFrame
dsales_weekly_sort_melt_vx = vaex.concat(melted)

# Ajustar o tipo da coluna 'week' (pode ser ajustado conforme a sua necessidade)
# Como você mencionou que 'week' deve estar presente, você pode precisar adicioná-lo aqui

# Exibir os tipos de dados
print(dsales_weekly_sort_melt_vx.dtypes)

# Verificando o resultado
print(dsales_weekly_sort_melt_vx.head())



id             string
item           string
sales         float16
store_code     string
department     string
region         string
store          string
dtype: object
  #  id                      item                sales  store_code    department    region        store
  0  ACCESORIES_1_001_NYC_1  ACCESORIES_1_001        0  NYC_1         ACCESORIES_1  New York      Greenwich_Village
  1  ACCESORIES_1_001_PHI_1  ACCESORIES_1_001        0  PHI_1         ACCESORIES_1  Philadelphia  Midtown_Village
  2  ACCESORIES_1_001_BOS_2  ACCESORIES_1_001        0  BOS_2         ACCESORIES_1  Boston        Roxbury
  3  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001        0  BOS_1         ACCESORIES_1  Boston        South_End
  4  ACCESORIES_1_001_PHI_2  ACCESORIES_1_001        0  PHI_2         ACCESORIES_1  Philadelphia  Yorktown
  5  ACCESORIES_1_001_BOS_3  ACCESORIES_1_001        0  BOS_3         ACCESORIES_1  Boston        Back_Bay
  6  ACCESORIES_1_001_NYC_2  ACCESORIES_1_001        0  NYC_2         

In [166]:
import vaex

# Carregar o DataFrame como Vaex
dsales_weekly_sort_vx = vaex.from_pandas(dsales_weekly_sort)

# Converter colunas categóricas para string (já que Vaex não suporta 'category')
categorical_columns = ['category', 'department', 'store', 'region', 'store_code']
for col in categorical_columns:
    dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('string')

# Converter colunas float64 para float32
float_columns = dsales_weekly_sort_vx.column_names
for col in float_columns:
    if dsales_weekly_sort_vx[col].dtype == 'float64':
        dsales_weekly_sort_vx[col] = dsales_weekly_sort_vx[col].astype('float32')  # Vaex não suporta float16

# Criar uma lista para armazenar os DataFrames temporários
melted = []
for week in range(1, 274):
    week_col = f'dweek_{week}'
    if week_col in dsales_weekly_sort_vx.column_names:  # Verificar se a coluna existe
        # Selecionar as colunas necessárias
        temp_df = dsales_weekly_sort_vx[['id', 'item', 'department', 'category','store', 'store_code', 'region', week_col]]
        
        # Adicionar a coluna de vendas
        temp_df['sales'] = temp_df[week_col]  # Cria a coluna 'sales' com os valores de 'week_col'
        
        # Adicionar a coluna 'week' com o nome da semana
        temp_df['week'] = week_col  # Cria a coluna 'week' com o nome da coluna de vendas

        # Remover a coluna original de vendas
        temp_df = temp_df.drop(columns=[week_col])  # Remove a coluna original

        # Adicionar o DataFrame temporário à lista melted
        melted.append(temp_df)

# Concatenar todos os DataFrames em um único DataFrame
if melted:
    dsales_weekly_sort_melt_vx = vaex.concat(melted)
else:
    print("Nenhuma coluna de vendas foi encontrada para derreter.")

# Exibir os tipos de dados
print(dsales_weekly_sort_melt_vx.dtypes)

# Verificando o resultado
print(dsales_weekly_sort_melt_vx.head())


id             string
item           string
sales         float16
week          float16
region         string
store          string
category       string
department     string
store_code     string
dtype: object
  #  id                      item                sales    week  region        store              category    department    store_code
  0  ACCESORIES_1_001_NYC_1  ACCESORIES_1_001        0       0  New York      Greenwich_Village  ACCESORIES  ACCESORIES_1  NYC_1
  1  ACCESORIES_1_001_PHI_1  ACCESORIES_1_001        0       0  Philadelphia  Midtown_Village    ACCESORIES  ACCESORIES_1  PHI_1
  2  ACCESORIES_1_001_BOS_2  ACCESORIES_1_001        0       0  Boston        Roxbury            ACCESORIES  ACCESORIES_1  BOS_2
  3  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001        0       0  Boston        South_End          ACCESORIES  ACCESORIES_1  BOS_1
  4  ACCESORIES_1_001_PHI_2  ACCESORIES_1_001        0       0  Philadelphia  Yorktown           ACCESORIES  ACCESORIES_1  PHI_2
  5  ACCE

In [167]:
import pandas as pd

# Exemplo: Carregando o DataFrame (substitua pelo seu DataFrame real)
# dsales_weekly_sort = pd.read_csv('seu_arquivo.csv')

# Usando o método melt do Pandas para transformar colunas dweek em linhas
dsales_weekly_sort_melt = pd.melt(dsales_weekly_sort, 
                                   id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], 
                                   value_vars=[f'dweek_{i}' for i in range(1, 274)], 
                                   var_name='week', 
                                   value_name='sales')

# Ajustar o tipo da coluna 'week' para string
dsales_weekly_sort_melt['week'] = dsales_weekly_sort_melt['week'].astype(str)

# Exibir os tipos de dados
print(dsales_weekly_sort_melt.dtypes)

# Verificando o resultado
print(dsales_weekly_sort_melt.head())


id             object
item           object
category       object
department     object
store          object
store_code     object
region         object
week           object
sales         float16
dtype: object
                       id              item    category    department  \
0  ACCESORIES_1_001_NYC_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
1  ACCESORIES_1_001_PHI_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
2  ACCESORIES_1_001_BOS_2  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
3  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
4  ACCESORIES_1_001_PHI_2  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   

               store store_code        region     week  sales  
0  Greenwich_Village      NYC_1      New York  dweek_1   0.00  
1    Midtown_Village      PHI_1  Philadelphia  dweek_1   0.00  
2            Roxbury      BOS_2        Boston  dweek_1   0.00  
3          South_End      BOS_1        Boston  dweek_1   0.00  
4           Yorktown      PHI

In [168]:
dsales_weekly_sort_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8323770 entries, 0 to 8323769
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   id          object 
 1   item        object 
 2   category    object 
 3   department  object 
 4   store       object 
 5   store_code  object 
 6   region      object 
 7   week        object 
 8   sales       float16
dtypes: float16(1), object(8)
memory usage: 523.9+ MB


In [169]:
 # Verificar o total de linhas no DataFrame dsales_weekly_sort
total_linhas_dsales_weekly_sort_melt = len(dsales_weekly_sort_melt)
print(f"Total de linhas em dsales_weekly_sort: {total_linhas_dsales_weekly_sort_melt}")

Total de linhas em dsales_weekly_sort: 8323770


In [170]:
# Verificando duplicatas
print(dsales_weekly_sort_melt[['item', 'category', 'department', 'store_code', 'region', 'week']].duplicated().sum())

0


In [172]:
df_merged_prices_cevents_sort.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7358227 entries, 0 to 1449053
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   item         object        
 1   category     object        
 2   store_code   object        
 3   yearweek     object        
 4   sell_price   float16       
 5   year         object        
 6   event        object        
 7   date         datetime64[ns]
 8   weekday      object        
 9   weekday_int  float64       
dtypes: datetime64[ns](1), float16(1), float64(1), object(7)
memory usage: 575.4+ MB


In [173]:
# Realizando o derretimento do DataFrame
dsales_weekly_sort_melt = pd.melt(dsales_weekly_sort, 
                                   id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], 
                                   value_vars=[f'dweek_{i}' for i in range(1, 274)], 
                                   var_name='week', 
                                   value_name='sales')

# Ajustar o tipo da coluna 'week' para string
dsales_weekly_sort_melt['week'] = dsales_weekly_sort_melt['week'].astype(str)

# Ordenar o DataFrame derretido
dsales_weekly_sort_melt = dsales_weekly_sort_melt.sort_values(by=['item', 'category','store_code']).reset_index(drop=True)     

# Exibir os tipos de dados
print(dsales_weekly_sort_melt.dtypes)

# Verificando o resultado
print(dsales_weekly_sort_melt.head())


id             object
item           object
category       object
department     object
store          object
store_code     object
region         object
week           object
sales         float16
dtype: object
                       id              item    category    department  \
0  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
1  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
2  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
3  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
4  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   

       store store_code  region     week  sales  
0  South_End      BOS_1  Boston  dweek_1   0.00  
1  South_End      BOS_1  Boston  dweek_2   0.00  
2  South_End      BOS_1  Boston  dweek_3   0.00  
3  South_End      BOS_1  Boston  dweek_4   0.00  
4  South_End      BOS_1  Boston  dweek_5   0.00  


In [174]:
total_linhas_ddsales_weekly_sort_melt = dsales_weekly_sort_melt.shape[0]
print(dsales_weekly_sort_melt)

                              id               item     category  \
0         ACCESORIES_1_001_BOS_1   ACCESORIES_1_001   ACCESORIES   
1         ACCESORIES_1_001_BOS_1   ACCESORIES_1_001   ACCESORIES   
2         ACCESORIES_1_001_BOS_1   ACCESORIES_1_001   ACCESORIES   
3         ACCESORIES_1_001_BOS_1   ACCESORIES_1_001   ACCESORIES   
4         ACCESORIES_1_001_BOS_1   ACCESORIES_1_001   ACCESORIES   
...                          ...                ...          ...   
8323765  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET   
8323766  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET   
8323767  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET   
8323768  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET   
8323769  SUPERMARKET_3_827_PHI_3  SUPERMARKET_3_827  SUPERMARKET   

            department          store store_code        region       week  \
0         ACCESORIES_1      South_End      BOS_1        Boston    dweek_1   
1         ACCESORIES_1      S

In [175]:
# Ordenando o DataFrame 'df_merged_prices_cevents' pela coluna 'item' em ordem ascendente
df_merged_prices_cevents_sort = df_merged_prices_cevents.sort_values(by=['item', 'category','store_code'], ascending= True)

# Exibindo o DataFrame ordenado
print(df_merged_prices_cevents_sort)

                      item     category store_code yearweek  sell_price  \
0         ACCESORIES_1_001   ACCESORIES      BOS_1   190000       10.98   
31800     ACCESORIES_1_001   ACCESORIES      BOS_1   201328       12.74   
31801     ACCESORIES_1_001   ACCESORIES      BOS_1   201328       12.74   
81062     ACCESORIES_1_001   ACCESORIES      BOS_1   201329       10.98   
105802    ACCESORIES_1_001   ACCESORIES      BOS_1   201330       10.98   
...                    ...          ...        ...      ...         ...   
4587264  SUPERMARKET_3_827  SUPERMARKET      PHI_3   201613        1.20   
4617754  SUPERMARKET_3_827  SUPERMARKET      PHI_3   201614        1.20   
4648244  SUPERMARKET_3_827  SUPERMARKET      PHI_3   201615        1.20   
4678734  SUPERMARKET_3_827  SUPERMARKET      PHI_3   201616        1.20   
4709224  SUPERMARKET_3_827  SUPERMARKET      PHI_3   201617        1.20   

            year            event       date  weekday  weekday_int  
0           1900  Evento Sem I

In [176]:
# fazendo o merge entre os dois dataframe 

In [177]:
df_merged_prices_cevents_sort.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7358227 entries, 0 to 4709224
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   item         object        
 1   category     object        
 2   store_code   object        
 3   yearweek     object        
 4   sell_price   float16       
 5   year         object        
 6   event        object        
 7   date         datetime64[ns]
 8   weekday      object        
 9   weekday_int  float64       
dtypes: datetime64[ns](1), float16(1), float64(1), object(7)
memory usage: 575.4+ MB


In [178]:
df_merged_prices_cevents_sort.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7358227 entries, 0 to 4709224
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   item         object        
 1   category     object        
 2   store_code   object        
 3   yearweek     object        
 4   sell_price   float16       
 5   year         object        
 6   event        object        
 7   date         datetime64[ns]
 8   weekday      object        
 9   weekday_int  float64       
dtypes: datetime64[ns](1), float16(1), float64(1), object(7)
memory usage: 575.4+ MB


In [179]:
import dask
print(dask.__version__)


2.11.0


In [180]:
import dask.dataframe as dd


In [ ]:
import dask.dataframe as dd
from dask.distributed import Client

# Iniciar um cliente Dask
client = Client()

# Converter DataFrames do Pandas para Dask DataFrames
dsales_weekly_sort_melt_dd = dd.from_pandas(dsales_weekly_sort_melt, npartitions=20)
df_merged_prices_cevents_sort_dd = dd.from_pandas(df_merged_prices_cevents_sort, npartitions=20)

# Ajustar tipos de dados para otimizar a memória
dsales_weekly_sort_melt_dd['category'] = dsales_weekly_sort_melt_dd['category'].astype('category')
df_merged_prices_cevents_sort_dd['category'] = df_merged_prices_cevents_sort_dd['category'].astype('category')

# Persistir o DataFrame de vendas se necessário
dsales_weekly_sort_melt_dd = dsales_weekly_sort_melt_dd.persist()

# Verificar colunas antes de dividir
print("Colunas no DataFrame dsales:", dsales_weekly_sort_melt_dd.columns)

# Dividir o DataFrame de preços em partes
df_parts = df_merged_prices_cevents_sort_dd.to_delayed()
df_final_list = []

# Iterar pelas partes para fazer o merge
for i, part in enumerate(df_parts):
    # Convertendo 'part' para um Dask DataFrame
    part_dd = dd.from_delayed(part)
    
    # Verificar colunas do DataFrame 'part' (apenas para a primeira parte)
    if i == 0:
        print("Colunas no DataFrame part:", part_dd.columns)

    # Realizar o merge
    df_temp = dd.merge(dsales_weekly_sort_melt_dd, part_dd, 
                       on=['item', 'category', 'store_code'], 
                       how='left')
    df_final_list.append(df_temp)

# Concatenar os resultados
df_final_dd = dd.concat(df_final_list, axis=0)

# Converter o resultado de volta para pandas
df_final = df_final_dd.compute()

# Verificar o resultado
print(df_final.head())


In [183]:
from dask.distributed import Client

# Criar o cliente Dask com limite de memória
client = Client(memory_limit='2GB')  # Ajuste o limite conforme sua máquina


In [184]:
# Tente um número menor de partições
n_partitions = 20  # ajuste conforme necessário
dsales_weekly_sort_melt_dd = dd.from_pandas(dsales_weekly_sort_melt, npartitions=n_partitions)
df_merged_prices_cevents_sort_dd = dd.from_pandas(df_merged_prices_cevents_sort, npartitions=n_partitions)



In [185]:
# Dividir manualmente o DataFrame em partes menores
dsales_parts = [dsales_weekly_sort_melt.iloc[i:i + 10000] for i in range(0, len(dsales_weekly_sort_melt), 10000)]
df_merged_parts = [df_merged_prices_cevents_sort.iloc[i:i + 10000] for i in range(0, len(df_merged_prices_cevents_sort), 10000)]

# Converter cada parte em um Dask DataFrame
dsales_dd_parts = [dd.from_pandas(part, npartitions=1) for part in dsales_parts]
df_merged_dd_parts = [dd.from_pandas(part, npartitions=1) for part in df_merged_parts]


In [186]:
# Antes de converter para Dask, otimize os tipos de dados
dsales_weekly_sort_melt['category'] = dsales_weekly_sort_melt['category'].astype('category')
df_merged_prices_cevents_sort['category'] = df_merged_prices_cevents_sort['category'].astype('category')


In [187]:
# Exemplo de merge e persistir o resultado
df_final_list = []
for part in df_merged_dd_parts:
    df_temp = dd.merge(dsales_dd_parts[0], part, on=['item', 'category', 'store_code'], how='left')
    df_final_list.append(df_temp.persist())


In [190]:
df_temp

,id,item,category,department,store,store_code,region,week,sales,yearweek,sell_price,year,event,date,weekday,weekday_int
npartitions=1,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,float16,object,float16,object,object,datetime64[ns],object,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [191]:
df_final_list

[Dask DataFrame Structure:
                    id    item category department   store store_code  region    week    sales yearweek sell_price    year   event            date weekday weekday_int
 npartitions=1                                                                                                                                                        
                object  object   object     object  object     object  object  object  float16   object    float16  object  object  datetime64[ns]  object     float64
                   ...     ...      ...        ...     ...        ...     ...     ...      ...      ...        ...     ...     ...             ...     ...         ...
 Dask Name: merge, 1 tasks,
 Dask DataFrame Structure:
                    id    item category department   store store_code  region    week    sales yearweek sell_price    year   event            date weekday weekday_int
 npartitions=1                                                                     

In [192]:
import dask.dataframe as dd

# Supondo que df_final_list seja sua lista de DataFrames Dask
df_final_dd = dd.concat(df_final_list)  # Concatena todos os DataFrames persistidos em um único DataFrame Dask

# Computa e converte para um DataFrame Pandas
df_final = df_final_dd.compute()

# Exibe o DataFrame final
print(df_final)


                          id              item    category    department  \
0     ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
1     ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
2     ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
3     ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
4     ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
...                      ...               ...         ...           ...   
9995  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9996  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9997  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9998  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9999  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   

          store store_code    region       week  sales yearweek  sell_price  \
0     So

In [194]:
print(df_final.head(25))  # Exibe as 5 primeiras linhas


                        id              item    category    department  \
0   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
1   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
2   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
3   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
4   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
5   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
6   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
7   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
8   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
9   ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
10  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
11  ACCESORIES_1_001_BOS_1  ACCESORIES_1_001  ACCESORIES  ACCESORIES_1   
12  ACCESORIES_1_001_BOS_1  ACCESORIES

In [195]:
print(df_final.tail(25))  # Exibe as 5 primeiras linhas

                          id              item    category    department  \
9975  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9976  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9977  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9978  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9979  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9980  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9981  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9982  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9983  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9984  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9985  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9986  ACCESORIES_1_004_NYC_4  ACCESORIES_1_004  ACCESORIES  ACCESORIES_1   
9987  ACCESO

In [196]:

# Supondo que df_final_dd seja o DataFrame Dask onde você deseja calcular o total de sell_price
# Calcule o total de sell_price, ignorando NaN
total_sell_price = df_final_dd['sell_price'].sum().compute()

# Se você quiser contar quantos NaN existem na coluna sell_price
count_nan = df_final_dd['sell_price'].isna().sum().compute()

# Exibindo os resultados
print(f"Total de sell_price (ignora NaN): {total_sell_price}")
print(f"Total de NaN em sell_price: {count_nan}")


Total de sell_price (ignora NaN): inf
Total de NaN em sell_price: 7350000


In [208]:
# Verificar estatísticas descritivas dos dados
print(df_final_dd['sell_price'].describe().compute())


count   2,132,174.00
mean             inf
std              NaN
min             1.25
25%             5.28
50%             5.28
75%             6.17
max            12.74
Name: sell_price, dtype: float64


In [210]:

# Verificar estatísticas descritivas dos dados
print(df_final_dd['sales'].describe().compute())


count   9,482,174.00
mean             inf
std              NaN
min             0.00
25%             0.00
50%             0.29
75%             0.86
max            10.43
Name: sales, dtype: float64


In [ ]:
# [9482174 rows x 16 columns] #  9482174

In [ ]:
 
# Total de sell_price sem NaN
total_sell_price = df_final_dd['sell_price'].count().compute()
print(total_sell_price)

# Total de sell_price considerando NaN como zero
total_sell_price_with_nan_as_zero = df_final_dd['sell_price'].fillna(0).count().compute()
print(total_sell_price_with_nan_as_zero)

# Calcular a porcentagem
if total_sell_price_with_nan_as_zero > 0:  # Evitar divisão por zero
    percentage = (total_sell_price/  total_sell_price_with_nan_as_zero ) * 100
else:
    percentage = float('nan')  # Ou 0, dependendo da sua preferência

# Exibindo resultados
print(f"Total de sell_price (ignora NaN): {total_sell_price}")
print(f"Total de sell_price (considerando NaN como 0): {total_sell_price_with_nan_as_zero}")
print(f"Porcentagem de sell_price em relação ao total (considerando NaN como 0): {percentage:.2f}%" if not pd.isna(percentage) else "Porcentagem: Indefinida")


In [216]:
 
# Total de sales sem NaN
total_sales = df_final_dd['sales'].count().compute()
print(total_sales)

# Total de sell_price considerando NaN como zero
total_sales_with_nan_as_zero = df_final_dd['sales'].fillna(999).count().compute()
print(total_sales_with_nan_as_zero)

# Calcular a porcentagem
if total_sales_with_nan_as_zero > 0:  # Evitar divisão por zero
    percentage_sales = (total_sales/  total_sales_with_nan_as_zero ) * 100
else:
    percentage_sales = float('nan')  # Ou 0, dependendo da sua preferência

# Exibindo resultados
print(f"Total de sales (ignora NaN): {total_sales}")
print(f"Total de sales (considerando NaN como 999): {total_sales_with_nan_as_zero}")
print(f"Porcentagem de sales em relação ao total (considerando NaN como 999): {percentage_sales:.2f}%" if not pd.isna(percentage_sales) else "Porcentagem: Indefinida")

9482174
9482174
Total de sales (ignora NaN): 9482174
Total de sales (considerando NaN como 999): 9482174
Porcentagem de sales em relação ao total (considerando NaN como 999): 100.00%


In [217]:
# Total de entradas em sell_price
total_entries = df_final_dd['sell_price'].count().compute() + df_final_dd['sell_price'].isna().sum().compute()

# Total de valores nulos em sell_price
total_nan = df_final_dd['sell_price'].isna().sum().compute()

# Calcular a porcentagem de NaN
if total_entries > 0:  # Evitar divisão por zero
    nan_percentage = (total_nan / total_entries) * 100
else:
    nan_percentage = float('nan')  # Ou 0, dependendo da sua preferência

# Exibindo resultados
print(f"Total de entradas em sell_price: {total_entries}")
print(f"Total de valores nulos em sell_price: {total_nan}")
print(f"Porcentagem de valores nulos em sell_price: {nan_percentage:.2f}%" if not pd.isna(nan_percentage) else "Porcentagem: Indefinida")


Total de entradas em sell_price: 9482174
Total de valores nulos em sell_price: 7350000
Porcentagem de valores nulos em sell_price: 77.51%


In [221]:
# Cálculo do total de nulos e total de elementos
total_nulos = df_final_dd.isnull().sum().sum().compute()
total_elements = df_final_dd.size.compute()  # Computar o tamanho do DataFrame

# Evitar divisão por zero
if total_elements > 0:
    percentual_total_nulos_df_final_dd = (total_nulos / total_elements) * 100
else:
    percentual_total_nulos_df_final_dd = float('nan')  # Ou 0, dependendo da sua preferência

# Exibindo o resultado
print(f'Percentual total de nulos: {percentual_total_nulos_df_final_dd:.2f}%' if not pd.isna(percentual_total_nulos_df_final_dd) else "Percentual total de nulos: Indefinido")



Percentual total de nulos: 33.91%
